In [51]:
import sys
import os
import mmh3
import numpy as np
import itertools
import collections
#################### Utilities ######################
#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 

################### Similarity ######################

docs = {} #dictionary mapping document id to document contents

datafolder = os.path.join("C:\\Users\\hasee\\Documents\\comptools\\week5\\DataWeek5", "ats_corpus_small")   # change to ats_corpus for large data set

for file in os.listdir(datafolder):
    filepath = os.path.join(datafolder, file)
    f = open(filepath, 'r')
    docs[file] = f.read()
    print("read document " + file)
    f.close()


read document calltounconv00baxt.txt
read document gospeltruth00whit.txt
read document lifeofrevrichard00baxt.txt
read document memoirjamesbrai00ricegoog.txt
read document practicalthought00nev.txt
read document remember00palm.txt
read document remembermeorholy00palm.txt
read document thoughtsonpopery00nevi.txt


In [52]:
''' Returns document as a list of hashes'''
'''
Creates shingles of size q, removing shingles of size < q. Removes duplicates and hashes the result.
'''
def hashed_lst_shingles(q, doc):
    
    doc = doc.split(" ")
    lst_shingles=[]

    lst_shingles = [doc[i:i+q] for i in range(0, len(doc), q-2)] # create list of shingles of length q

    lst_shingles = [x for x in lst_shingles if len(x)==q] # remove shingles with length < q

    seed=0
    lst_shingles_h = [listhash(shingle, seed) for shingle in lst_shingles] # hash the singles

    lst_shingles_h = list(set(lst_shingles_h)) # remove duplicates

    return lst_shingles_h

# doc = "You and me, we made a vow. For better or for worse. I can't believe you let me down"
# lst_shnigles = hashed_lst_shingles(3, doc)
# print(lst_shnigles)

### MinHashing implementation

In [53]:
def generate_matrices(docs, shingle_size=3, num_hash_funcs=2):
    
    doc_hashes = [hashed_lst_shingles(shingle_size, docs[key]) for key in docs.keys()] # returns list of lists, where each elem contains hash list for that document.

    my_elements = list(itertools.chain(*doc_hashes)) # flatten to get a single list
    my_elements = list(set(my_elements)) # set() to remove duplicates. This gives the universe.

    ''' Iterate over documents (S1, S2, etc.).
        insert 1 if element from universe exists in the doc else 0
        finally, concatenate all lists to get matrix m
    '''
    boolean_m = []

    for my_doc in doc_hashes:
        my_doc=set(my_doc) # Without this, the code is too slow to run!
        boolean_m.append([1 if element in my_doc else 0 for element in my_elements])

    m = np.array(boolean_m).T # Matrix m!
   
    ''' Compute hashes to get matrix h_m'''
    rows = np.arange(len(my_elements))
    
    hashes_lst = []
    for i in range(num_hash_funcs):
        hashes_lst.append([mmh3.hash(x, seed=i) for x in rows])

    h_m = np.array(hashes_lst).T # Matrix h_m!
    

    return m, h_m

def min_hash_alg(m, h_m):
    sig_m = np.full((h_m.shape[1], m.shape[1]), np.inf) # Signature matrix. This stores the final result!

    my_args = np.argwhere(m == 1)
    for args in my_args:
        row=args[0]
        column=args[1]
        for i in range(h_m.shape[1]):
            if h_m[row][i] < sig_m[i, column]:
                sig_m[i, column] = h_m[row][i]
    
    return sig_m


In [54]:
test_docs={}
test_docs["one"]="You and me, we made a vow. For better or for worse. I can't believe you let me down"
test_docs["two"]="Time, space and state. Equal everything explanable."
test_docs["three"]="You and me, we made a vow. For better or for worse. I can't believe you let me down"
m, h_m = generate_matrices(test_docs, shingle_size=3, num_hash_funcs=2)
sig_m = min_hash_alg(m, h_m)
sig_m

ssss


array([[-1.89304750e+09, -1.19706312e+09, -1.89304750e+09],
       [-7.78479335e+08, -2.07294148e+09, -7.78479335e+08]])

From results of sig matrix above, documents 1 and 3 are very similar (similar hashes)

In [55]:
''' Testing own implementation with input example from book'''
m = np.array([[1,0,0,1], [0,0,1,0], [0,1,0,1], [1,0,1,1],[0,0,1,0]])
h_m = np.array([[1,1],[2,4],[3,2],[4,0],[0,3]])
print("Signature matrix:")
sig_m = min_hash_alg(m, h_m)
sig_m

Signature matrix:


array([[1., 3., 0., 1.],
       [0., 2., 0., 0.]])

In [56]:
m, h_m = generate_matrices(docs, shingle_size=3, num_hash_funcs=100)

sig_m = min_hash_alg(m, h_m)
sig_m

ssss


array([[-2.14747874e+09, -2.14747580e+09, -2.14728504e+09,
        -2.14746990e+09, -2.14746630e+09, -2.14707323e+09,
        -2.14665401e+09, -2.14746630e+09],
       [-2.14746538e+09, -2.14724729e+09, -2.14715192e+09,
        -2.14748198e+09, -2.14744212e+09, -2.14738709e+09,
        -2.14715744e+09, -2.14743367e+09],
       [-2.14743896e+09, -2.14735663e+09, -2.14743896e+09,
        -2.14743896e+09, -2.14745845e+09, -2.14659046e+09,
        -2.14659046e+09, -2.14743182e+09],
       [-2.14746356e+09, -2.14741023e+09, -2.14742918e+09,
        -2.14746529e+09, -2.14744603e+09, -2.14743953e+09,
        -2.14743953e+09, -2.14744603e+09],
       [-2.14748073e+09, -2.14688307e+09, -2.14748073e+09,
        -2.14744967e+09, -2.14745262e+09, -2.14513463e+09,
        -2.14513463e+09, -2.14745262e+09],
       [-2.14747971e+09, -2.14737424e+09, -2.14746920e+09,
        -2.14745471e+09, -2.14747544e+09, -2.14745785e+09,
        -2.14745785e+09, -2.14743593e+09],
       [-2.14747919e+09, -2.147369

In [57]:
sig_m

array([[-2.14747874e+09, -2.14747580e+09, -2.14728504e+09,
        -2.14746990e+09, -2.14746630e+09, -2.14707323e+09,
        -2.14665401e+09, -2.14746630e+09],
       [-2.14746538e+09, -2.14724729e+09, -2.14715192e+09,
        -2.14748198e+09, -2.14744212e+09, -2.14738709e+09,
        -2.14715744e+09, -2.14743367e+09],
       [-2.14743896e+09, -2.14735663e+09, -2.14743896e+09,
        -2.14743896e+09, -2.14745845e+09, -2.14659046e+09,
        -2.14659046e+09, -2.14743182e+09],
       [-2.14746356e+09, -2.14741023e+09, -2.14742918e+09,
        -2.14746529e+09, -2.14744603e+09, -2.14743953e+09,
        -2.14743953e+09, -2.14744603e+09],
       [-2.14748073e+09, -2.14688307e+09, -2.14748073e+09,
        -2.14744967e+09, -2.14745262e+09, -2.14513463e+09,
        -2.14513463e+09, -2.14745262e+09],
       [-2.14747971e+09, -2.14737424e+09, -2.14746920e+09,
        -2.14745471e+09, -2.14747544e+09, -2.14745785e+09,
        -2.14745785e+09, -2.14743593e+09],
       [-2.14747919e+09, -2.147369

In [58]:
''' Jaccard similarity'''
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

score = jaccard(set(sig_m[:, 5]), set(sig_m[:, 6])) # finding similarity b/w documents 5 and 6. remember00palm.txt and remembermeorholy00palm.txt

print(score)

0.6129032258064516


We see that docs 5 and 6 are quite similar (0.61).

### LSH implementation

In [66]:
''' Implementation of LSH, dividing signature matrix into b band with r rows each'''
def LSH(b, r):
    b=20
    r=5
    #b*r=num_hash_funcs

    sim_hashes=[]
    start=0
    for i in range(b):
        sim_hashes.append([listhash(col, seed=i) for col in sig_m[start:start+r,:].T])
        start=i+r

    return sim_hashes

''' Find candidate pairs by checking to see if the hashes match.
Then we check to see that the Jaccard similarity b/w each pair of docs is atleast t. If so, we consider it a candidate pair otherwise not '''
def get_cand_pairs(sim_hashes, t):
    cand_pairs=set()
    for L in sim_hashes:
        dups = collections.defaultdict(list)
        for i, e in enumerate(L):
            dups[e].append(i)
        for k, v in sorted(dups.items()):
            if len(v) >= 2:
                cand_pairs.add(tuple(v))
                # print(k, v)
    cand_pairs=list(cand_pairs)
    filtered_cand_pairs = [pair for pair in cand_pairs if (jaccard(set(sig_m[:, pair[0]]), set(sig_m[:, pair[1]])) > t)]
    return filtered_cand_pairs
    # return cand_pairs

In [67]:
b=20
r=5
sim_hashes = LSH(b=20, r=5)
pairs=get_cand_pairs(sim_hashes, t=(1/b)**(1/r))
pairs

[(5, 6)]

As can be seen using LSH, we find that documents 5 and 6 are similar.

### Development Junk

In [64]:
X = np.random.rand(10,5)

In [65]:
for col in X[0:3, :].T:
    print(col)

[0.76717558 0.16896812 0.60805319]
[0.10332414 0.71198539 0.31250171]
[0.08204225 0.43062834 0.52636287]
[0.94627444 0.85569538 0.23878093]
[0.5296346  0.48514638 0.3348344 ]
